In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import json
import ast

from sklearn.svm import SVC
from skimage import io
from skimage.color import rgb2gray

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline 

# to convert the pixels:
from google.colab.patches import cv2_imshow
import cv2

In [2]:
cat_breed_url = "https://raw.githubusercontent.com/AliceLiu17/Cat-Breed-Image-Classification/main/data/cats_data.csv"
df = pd.read_csv(cat_breed_url)
df.head()

,Unnamed: 0,id,url,type,age,gender,size,coat,breed,photos,med_photos
0,0,46744842,https://www.petfinder.com/cat/arianna-46744842...,Cat,Adult,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
1,1,46717321,https://www.petfinder.com/cat/ozzy-46717321/oh...,Cat,Adult,Male,Extra Large,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
2,2,46626338,https://www.petfinder.com/cat/zena-46626338/ca...,Cat,Senior,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
3,3,46620170,https://www.petfinder.com/cat/mika-46620170/ca...,Cat,Baby,Female,Small,Short,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...
4,4,46582751,https://www.petfinder.com/cat/ruby-46582751/fl...,Cat,Young,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...


In [3]:
! pip install jsbeautifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74 kB 1.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for jsbeautifier: filename=jsbeautifier-1.14.7-py3-none-any.whl size=93578 sha256=d620a1bb72aa6433034523125fea2fedb8e048c5f313bda5dda1ab4e1f44a8d1
  Stored in directory: /root/.cache/pip/wheels/cb/34/4f/52898647909379a8ca3011e23388f3666795d8f851597320ea
Successfully built jsbeautifier


In [4]:
def strip_json_image_link(input_string, size_you_want='small'):
  the_json = ast.literal_eval(input_string)[0]
  link = the_json[size_you_want]
  return(link)


df['link'] = df["photos"].apply(strip_json_image_link)
df.head()

,Unnamed: 0,id,url,type,age,gender,size,coat,breed,photos,med_photos,link
0,0,46744842,https://www.petfinder.com/cat/arianna-46744842...,Cat,Adult,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...
1,1,46717321,https://www.petfinder.com/cat/ozzy-46717321/oh...,Cat,Adult,Male,Extra Large,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...
2,2,46626338,https://www.petfinder.com/cat/zena-46626338/ca...,Cat,Senior,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...
3,3,46620170,https://www.petfinder.com/cat/mika-46620170/ca...,Cat,Baby,Female,Small,Short,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...
4,4,46582751,https://www.petfinder.com/cat/ruby-46582751/fl...,Cat,Young,Female,Medium,NaN,Abyssinian,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,['https://dl5zpyw5k3jeb.cloudfront.net/photos/...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...


In [5]:
df["link"].iloc[0]

'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46744842/1/?bust=1575403984&width=100'

In [9]:
# convert the dataframe into a list
link_list = df["link"].values.tolist()

In [10]:
def a_function(input_url):
  try:
    resp = urllib.request.urlopen(input_url)
    print("this worked:", input_url)
  except:
    print("this is bad url and above code didn't work:", input_url)
    resp = None
  return resp

a_function("https://www.google.com")
a_function("www.akjsdfasd;lfjasdfjkl.com")

this worked: https://www.google.com
this is bad url and above code didn't work: www.akjsdfasd;lfjasdfjkl.com


for url in link_list:
  a_function(url)

In [12]:
df.shape

(67145, 12)

In [13]:
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/41384239/1/?bust=1523505638&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/20935636/1/?bust=1445973066&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/45999734/1/?bust=1573780012&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44643500/1/?bust=1557326883&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44284574/1/?bust=1553187658&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44228106/1/?bust=1552567088&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44038298/1/?bust=1550413571&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/42664207/2/?bust=1536820105&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46559328/1/?bust=1573615912&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46590712/1/?bust=1573862304&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46494368/1/?bust=1573060470&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46003926/1/?bust=1573173138&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46003701/1/?bust=1570026695&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46002084/1/?bust=1572566857&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/45609859/6/?bust=1565793008&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/45173058/5/?bust=1562944195&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/43629146/4/?bust=1545882011&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44467139/1/?bust=1574769924&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/29872984/4/?bust=1563675170&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46671890/1/?bust=1575447604&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46531959/6/?bust=1574554884&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46746860/1/?bust=1575413469&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46744633/1/?bust=1575402279&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/25183447/1/?bust=1358868481&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/35183025/1/?bust=1463945914&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/34815717/1/?bust=1459689563&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/20418641/1/?bust=1311856578&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/20418611/1/?bust=1311856578&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44840019/1/?bust=1559304081&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46601479/1/?bust=1573962718&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46329795/1/?bust=1573200593&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/45777764/4/?bust=1574426954&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46705875/3/?bust=1575340717&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46700115/2/?bust=1574874506&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46497049/1/?bust=1573072114&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/45309355/3/?bust=1563558548&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/46652656/4/?bust=1575342099&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/45322069/5/?bust=1564259655&width=100'].index, inplace=True)
df.drop(df[df['link'] == 'https://dl5zpyw5k3jeb.cloudfront.net/photos/pets/44384207/1/?bust=1554381199&width=100'].index, inplace=True)


In [14]:
df.shape

(67106, 12)

### Convert in CSV

In [ ]:
from google.colab import files
df.to_csv('cat_clean.csv')
files.download("cat_clean.csv")

### DIRTY STUFF DOWN THERE--IGNORE

In [ ]:
### Finding bad URL

# def find_bad_link(url):
#   try:
#     resp = urllib.request.urlopen(url)
#     image = np.asarray(bytearray(resp.read()), dtype = "uint8")
#     image = cv2.imdecode(image, cv2.IMREAD_COLOR)
#   except:
#     print(url, '= THIS IS A BAD URL')
#     image = None
#   return image

In [11]:
# iterate through the list and feed it into the function
# x = []
# for url in link_list:
#   x.append(url_to_array(url))

# df_array = pd.DataFrame({'link_array':x})
# df_array.head()

In [ ]:
# # convert the dataframe into a list
# link_list = df["link"].values.tolist()
# label_list = df['breed'].values.tolist()


# # just get 10 to see if new function works
# link_list_test = link_list[:1000]
# label_list_test = label_list[:1000]

In [ ]:
# # iterate through the list and feed it into the function
# x = []
# for label, url in zip(label_list_test, link_list_test):
#   pixel_array = url_to_array(url)
#   x.append( [pixel_array, label] )

# df_array = pd.DataFrame(x)
# df_array.columns = ['image_array', 'label']

# print(df_array.shape)
# df_array = df_array.dropna()
# print(df_array.shape)

# df_array.head()

In [ ]:
# df["link"] = df["link"].apply(find_bad_link)

In [ ]:
# df['link'] = df['link'].apply(url_to_array)